## Parameter correlation figure matrices
This notebook provides pairwise correlation plots for groups or parameters by default. It also provides an easy interface for plotting bivariate distributions of user-specified combinations of parameters from the calibration algorithm.

In [ ]:
import arviz as az
import numpy as np
from arviz.labels import MapLabeller
import matplotlib as mpl

from aust_covid.constants import RUN_IDS, PRIMARY_ANALYSIS, BURN_IN, set_project_base_path
from emutools.utils import load_param_info

project_paths = set_project_base_path("../")
RUNS_PATH = project_paths["RUNS_PATH"]

az.rcParams["plot.max_subplots"] = 200
mpl.rcParams["axes.facecolor"] = (0.2, 0.2, 0.4)

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / "output/calib_full_out.nc")
idata = idata.sel(draw=np.s_[BURN_IN:])
abbreviations = load_param_info()["abbreviations"]

In [ ]:
epi_params = [param for param in idata.posterior.keys() if "_dispersion" not in param]

key_params = [
    "contact_rate", 
    "latent_period",
    "infectious_period", 
    "natural_immunity_period", 
    "start_cdr", 
    "imm_infect_protect",
    "ba2_escape",
    "ba5_escape",
    "imm_prop",
]

imm_params = [
    "natural_immunity_period",
    "imm_infect_protect",
    "ba2_escape",
    "ba5_escape",
    "imm_prop",
]

### Epidemiological parameters

In [ ]:
az.plot_pair(idata, var_names=epi_params, kind="kde", textsize=35, labeller=MapLabeller(var_name_map=abbreviations));

### Selected "key" parameters
Adjustment to axes to more clearly visualise high-density regions - to produce Figure 6 of manuscript.

In [ ]:
fig = az.plot_pair(idata, var_names=key_params, kind="kde", textsize=30, labeller=MapLabeller(var_name_map=abbreviations))
fig[2][0].set_ylim((0.0, 300.0))
fig[3][3].set_xlim((0.0, 300.0));

### Immunity-related parameters

In [ ]:
az.plot_pair(idata, var_names=imm_params, kind="kde", textsize=20, labeller=MapLabeller(var_name_map=abbreviations));